In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder

import os, shutil
import torch.nn.functional as F

import torch.optim as optim
from torchvision import datasets, transforms, utils

import numpy as np
import matplotlib.pyplot as plt

import torchvision
from torchvision.models import vgg16

from itertools import islice


In [13]:
original_dataset_dir = 'C:/Users/pauli/Documents/M2/ML et DL/projet/Git/DamageDetection'

In [14]:
train_damage_dir = original_dataset_dir + '/train_another/damage'
validation_damage_dir = original_dataset_dir + '/validation_another/damage'
test_damage_dir = original_dataset_dir + '/test/damage'

train_nodamage_dir = original_dataset_dir + '/train_another/no_damage'
validation_nodamage_dir = original_dataset_dir + '/validation_another/no_damage'
test_nodamage_dir = original_dataset_dir + '/test/no_damage'

In [15]:
print('total training damage images: ',len(os.listdir(train_damage_dir)))
print('total validation damage images: ',len(os.listdir(validation_damage_dir)))
print('total test damage images: ',len(os.listdir(test_damage_dir)))

print('total training no damage images: ',len(os.listdir(train_nodamage_dir)))
print('total validation no damage images: ',len(os.listdir(validation_nodamage_dir)))
print('total test no damage images: ',len(os.listdir(test_nodamage_dir)))

total training damage images:  5000
total validation damage images:  1000
total test damage images:  1000
total training no damage images:  5000
total validation no damage images:  1000
total test no damage images:  1000


In [16]:
   
    
    
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, (3,3))
        self.pool1 = nn.MaxPool2d((2,2))
        self.conv2 = nn.Conv2d(32, 64, (3,3))
        self.pool2 = nn.MaxPool2d((2,2))
        self.conv3 = nn.Conv2d(64, 128, (3,3))
        self.pool3 = nn.MaxPool2d((2,2))
        self.conv4 = nn.Conv2d(128, 128, (3,3))
        self.pool4 = nn.MaxPool2d((2,2))
        self.fc1 = nn.Linear(128 * 7 * 7, 512)
        self.fc2 = nn.Linear(512, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool2(x)
        x = F.relu(self.conv4(x))
        x = self.pool3(x)
        x = x.view(-1, 128 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x
    
    

In [17]:
model = CNN()

In [18]:
import torchsummary
model = CNN()
torchsummary.summary(model, (3,150,150))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 148, 148]             896
         MaxPool2d-2           [-1, 32, 74, 74]               0
            Conv2d-3           [-1, 64, 72, 72]          18,496
         MaxPool2d-4           [-1, 64, 36, 36]               0
            Conv2d-5          [-1, 128, 34, 34]          73,856
         MaxPool2d-6          [-1, 128, 17, 17]               0
            Conv2d-7          [-1, 128, 15, 15]         147,584
         MaxPool2d-8            [-1, 128, 7, 7]               0
            Linear-9                  [-1, 512]       3,211,776
           Linear-10                    [-1, 1]             513
Total params: 3,453,121
Trainable params: 3,453,121
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.26
Forward/backward pass size (MB): 11.53
Params size (MB): 13.17
Estim

In [19]:
#criterion = nn.BCELoss() #torch.nn.functional.binary_cross_entropy()
#optimizer = optim.RMSprop(model.parameters(), lr=1e-4)
criterion = nn.BCELoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.0001)

In [20]:
train_datagen = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    #transforms.Normalize((0,), (1/255,))
])

train_dataset = ImageFolder('C:/Users/pauli/Documents/M2/ML et DL/projet/Git/DamageDetection/train_another', transform=train_datagen)
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)

validation_datagen = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    #transforms.Normalize((0,), (1/255,))
])

val_dataset = ImageFolder('C:/Users/pauli/Documents/M2/ML et DL/projet/Git/DamageDetection/validation_another', transform=validation_datagen)
val_loader = DataLoader(val_dataset, batch_size=20, shuffle=True)


In [21]:
print('Found {} images belonging to {} classes.'.format(len(train_dataset), len(train_dataset.classes)))
print('Found {} images belonging to {} classes.'.format(len(val_dataset), len(val_dataset.classes)))

Found 10000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:

criterion = nn.BCELoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.0001)

# Initialisation des paramètres d'entraînement
steps_per_epoch = 100
epochs = 50
validation_steps = 50

loss_values = []

def fit(model, train_loader, val_loader, epochs, steps_per_epoch, validation_steps):
    for epoch in range(epochs):
        accuracy = 0.0
        running_loss = 0.0
        # Entraîne le modèle sur les données d'entraînement
        model.train()
        for i, (images, labels) in enumerate(train_loader):
            # Entraîne le modèle sur un lot de données
            outputs = model(images)
            loss = criterion(outputs, labels.float().view(-1,1))
            loss_values.append(loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            #running_loss += loss.item()
            accuracy += (torch.round(outputs) == labels.float().view(-1,1)).sum().item() / len(labels)
            running_loss += loss.item()
        
            if (i+1) % steps_per_epoch == 0:
                break
        train_loss = running_loss / steps_per_epoch
        train_acc = accuracy / steps_per_epoch 
                
        
        model.eval()
        with torch.no_grad():
            val_loss = 0
            val_acc = 0
            for i, (images, labels) in enumerate(val_loader):
                outputs = model(images)
                val_loss += criterion(outputs, labels.float().view(-1,1)).item()
                _, predicted = torch.max(outputs.data, 1)
                val_acc += (torch.round(outputs) == labels.float().view(-1,1)).sum().item() / len(labels)
                
                if i >= validation_steps:
                    break
        
        val_loss /= validation_steps
        val_acc /= validation_steps 

        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {loss.item():.4f}, Train Accuracy: {train_acc:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

# Lancer l'entraînement
fit(model, train_loader, val_loader, epochs, steps_per_epoch, validation_steps)



Epoch [1/50], Train Loss: 0.3465, Train Accuracy: 0.6880, Validation Loss: 0.5962, Validation Accuracy: 0.7150
Epoch [2/50], Train Loss: 0.2742, Train Accuracy: 0.8495, Validation Loss: 0.3180, Validation Accuracy: 0.9050
Epoch [3/50], Train Loss: 0.3879, Train Accuracy: 0.8660, Validation Loss: 0.2508, Validation Accuracy: 0.9220
Epoch [4/50], Train Loss: 0.1956, Train Accuracy: 0.8835, Validation Loss: 0.2429, Validation Accuracy: 0.9130
Epoch [5/50], Train Loss: 0.6793, Train Accuracy: 0.8970, Validation Loss: 0.2817, Validation Accuracy: 0.9100
Epoch [6/50], Train Loss: 0.1135, Train Accuracy: 0.9020, Validation Loss: 0.2132, Validation Accuracy: 0.9420
Epoch [7/50], Train Loss: 0.1039, Train Accuracy: 0.9090, Validation Loss: 0.2694, Validation Accuracy: 0.9040
Epoch [8/50], Train Loss: 0.2670, Train Accuracy: 0.9110, Validation Loss: 0.2113, Validation Accuracy: 0.9420
Epoch [9/50], Train Loss: 0.2011, Train Accuracy: 0.9170, Validation Loss: 0.2035, Validation Accuracy: 0.9410
E

In [ ]:


step = np.linspace(0, 100, 10500)

fig, ax = plt.subplots(figsize=(8,5))
plt.plot(step, np.array(loss_values))
plt.title("Step-wise Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()



In [ ]:

plt.plot(epoch_nums, training_acc, 'bo', label = 'Training acc')
plt.plot(epoch_nums, validation_acc, 'r', label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epoch_nums, training_loss, 'bo', label = 'Training loss')
plt.plot(epoch_nums, validation_loss, 'r', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
import torchvision.utils as vutils

layer_names = []
for i in range(8):
    layer_names.append("layer_{}".format(i))
images_per_row = 16

for layer_name, layer_activation in zip(layer_names, activations):
    n_features = layer_activation.shape[1]
    size = layer_activation.shape[2]
    n_cols = n_features // images_per_row
    display_grid = torch.zeros((n_cols * size, images_per_row * size))
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0, col * images_per_row + row, :, :]
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = channel_image.clamp(0, 255).to('uint8')
            display_grid[col * size:(col + 1) * size, row * size:(row + 1) * size] = channel_image
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(vutils.make_grid(display_grid, normalize=False, scale_each=False), cmap='viridis')
